In [1]:
import torch
import torch.nn as nn
from functorch import vmap
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F
from itertools import accumulate
import cvxpy as cp
import gurobipy
import numpy as np
from torch.autograd.functional import hessian
import torch
from torch.autograd import Function, Variable
from torch.nn import Module
from torch.nn.parameter import Parameter
import util
from collections import OrderedDict
import plotly.graph_objs as go
import random
import matplotlib.pyplot as plt
import pandas as pd
from baseline import MLP, RNNmodel
import keras
torch.set_default_tensor_type(torch.DoubleTensor)
torch.manual_seed(0)

/Users/bb/opt/anaconda3/envs/dr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class ICNN(torch.nn.Module):
    """Input Convex Neural Network"""
    def __init__(self, input_num=6, hidden_num=24):
        super().__init__()

        self.linear_z1 = nn.Linear(hidden_num, 1, bias=False)
        self.linear_y0 = nn.Linear(input_num, hidden_num)
        self.linear_y1 = nn.Linear(input_num, 1)
        torch.nn.init.uniform_(self.linear_z1.weight, a=1e-2,b=4)
        torch.nn.init.uniform_(self.linear_y0.weight, a=-2,b=2)
        #self.linear_y0.weight  = torch.nn.Parameter(torch.tril(torch.ones(24,24)))
        #self.linear_z1.weight.data.fill_(1.0)
        self.act = nn.Softplus()

    def forward(self, y):
        z1 = self.act(self.linear_y0(y))
        z = self.act(self.linear_z1(z1) + self.linear_y1(y))
        return z

# Hessian: [T,B,1,1]
# q: [T,B,1]
torch.manual_seed(0)
Cf = ICNN()
B, T = 50, 6
y = torch.rand(B,T)
y2 = torch.rand(B,T)
with torch.enable_grad():
    tau = y.data
    #tau2 = y2.data
    #tau = torch.concat([y,y2],dim=1)
    tau = Variable(tau, requires_grad=True)
    cost = Cf(tau)
    grad = torch.autograd.grad(cost.sum(), tau, create_graph=True, retain_graph=True)[0]
    print('grad', grad.shape)
    hessian = list()
    for i in range(T):
        hessian.append(
            torch.autograd.grad(grad[:,i].sum(), tau,
                                retain_graph=True)[0]
        )
    hessian = torch.stack(hessian, dim=-1)
hessian[0][:5,:5], grad[0]

grad torch.Size([50, 6])


(tensor([[ 6.6757, -1.1579,  0.6692, -2.8934,  1.1456],
         [-1.1579,  9.6575,  0.9183,  1.2770, -4.2600],
         [ 0.6692,  0.9183,  5.1384, -0.1494, -1.5511],
         [-2.8934,  1.2770, -0.1494,  9.6883, -1.3674],
         [ 1.1456, -4.2600, -1.5511, -1.3674, 10.1102]]),
 tensor([-0.3840,  1.6371,  8.4603,  6.0742, -6.2592, 18.9336],
        grad_fn=<SelectBackward0>))

In [ ]:
y = torch.rand(B,T)
y2 = torch.rand(B,T)
torch.concat([y,y2],dim=1).shape

In [8]:
class DRagent(nn.Module):
    """
    Using this layer, we train c1, c2, E1, E2, and eta, other parameters are infered from historical data.
    """
    def __init__(self, P1, P2, T):
        super().__init__()

        self.E1 = nn.Parameter(0.5 * torch.ones(1))
        self.E2 = nn.Parameter(-0.5 * torch.ones(1))
        self.eta = nn.Parameter(0.9 * torch.ones(1))#0.89 * torch.ones(1) #nn.Parameter(0.9 * torch.ones(1))
        self.T = T
        eps = 1e-5

        obj = (lambda d, p, price, c1, c2, E1, E2, eta: -price @ (d - p) + c1 @ d  +  cp.QuadForm(d, c2)
            if isinstance(d, cp.Variable) #cp.sum_squares(cp.sqrt(cp.diag(c2)) @ d)
            else -price @ (d - p) + c1 @ d  + d @ c2 @ d #torch.sum((torch.sqrt(torch.diag(c2)) @ d)**2)
        )
        self.objective = obj
        # self.costNN = nn.Sequential(OrderedDict([('linear1',nn.Linear(1, 24)),
        #             ('silu1',nn.SiLU()),
        #             ('linear2',nn.Linear(24, 1))]))
        # torch.nn.init.uniform_(self.costNN.linear1.weight, a=-2,b=-1e-2)
        # torch.nn.init.uniform_(self.costNN.linear2.weight, a=1e-2, b=2)
        # self.costNN.linear1.bias.data.fill_(0)
        # self.costNN.linear2.bias.data.fill_(0)
        self.costNN = ICNN()

        self.ineq1 = (lambda d, p, price, c1, c2, E1, E2, eta: p - torch.ones(T, dtype=torch.double) * P1)
        self.ineq2 = (lambda d, p, price, c1, c2, E1, E2, eta: torch.ones(T, dtype=torch.double)* 0 - p)
        self.ineq3 = (lambda d, p, price, c1, c2, E1, E2, eta: d - torch.ones(T, dtype=torch.double) * P2)
        self.ineq4 = (lambda d, p, price, c1, c2, E1, E2, eta: torch.ones(T, dtype=torch.double) * 0 - d)
        self.ineq5 = (lambda d, p, price, c1, c2, E1, E2, eta: torch.tril(torch.ones(T, T, dtype=torch.double)) @ (eta * p - d / eta)
            - torch.as_tensor(np.arange(eps, (T+1)*eps, eps))
            - torch.ones(T, dtype=torch.double) * E1)
        self.ineq6 = lambda d, p, price, c1, c2, E1, E2, eta: torch.ones(
            T, dtype=torch.double
        ) * E2 - torch.tril(torch.ones(T, T, dtype=torch.double)) @ (eta * p - d / eta) + torch.as_tensor(np.arange(eps, (T+1)*eps, eps))
        
        self.record = []
        self.layer = util.OptLayer(
            [cp.Variable(T), cp.Variable(T)],
            [
                cp.Parameter(T,),
                cp.Parameter(T),
                cp.Parameter((T,T), PSD=True),
                cp.Parameter(1),
                cp.Parameter(1),
                cp.Parameter(1),
            ],
            obj,
            [self.ineq1, self.ineq2, self.ineq3, self.ineq4, self.ineq5, self.ineq6],
            [],
            solver="GUROBI",
            verbose=False,
        )
    
    def approximate_cost(self, y, diff=True):
        # y: demand response [T,B,1]
        Cf = self.costNN
        with torch.enable_grad():
            tau = y.data
            #tau2 = p.data
            #tau = torch.concat([tau,tau2],axis=1)
            tau = Variable(tau, requires_grad=True)
            cost = Cf(tau)
            grad = torch.autograd.grad(cost.sum(), tau, create_graph=True, retain_graph=True)[0] # [B,T]
            hessian = list()
            for i in range(self.T):
                hessian.append(torch.autograd.grad(grad[:,i].sum(), tau, retain_graph=True)[0])
            hessian = torch.stack(hessian, dim=-1) # [B,T,T]
            # if not diff:
            #     return hessian.data, grad.data
            # return hessian, grad
            if not diff:
                return hessian[:,:self.T,:].data, grad[:,:self.T].data
            return hessian[:,:self.T,:], grad[:,:self.T]

    
    def dr_solver(self, *batch_params):
        variables = [cp.Variable(T), cp.Variable(T)]
        parameters = [cp.Parameter(T,), cp.Parameter(T), cp.Parameter(T), cp.Parameter(1), cp.Parameter(1), cp.Parameter(1),]
        problem = cp.Problem(cp.Minimize(self.objective(*variables, *parameters)), [ineq(*variables, *parameters) <= 0 for ineq in [self.ineq1, self.ineq2, self.ineq3, self.ineq4, self.ineq5, self.ineq6]])
        for batch in range(batch_params[0].shape[0]):
            print(batch)
            # solve the optimization problem
            params = [p[batch] for p in batch_params]
            with torch.no_grad():
                for i, p in enumerate(parameters):
                    p.value = params[i].double().numpy()
                problem.solve(solver='GUROBI')
                z = [torch.tensor(v.value).type_as(params[0]) for v in variables]
                print(z[0].shape)


    def forward(self, price, dr, ite):
        # price: [B,T]
        # dr: [B, T]
        if ite < 500:
            d = torch.rand(price.shape[0], price.shape[1]) # [B,T]
            p = torch.rand(price.shape[0], price.shape[1])
        else:
            d = dr.data
        #figure, axes = plt.subplots(1,1,figsize=(8,4))
        for i in range(20):
            #print(i)
            c2, c1 = self.approximate_cost(d,  diff=False)
            d, p = self.layer(price, 
                            c1,  c2,
                            self.E1.expand(price.shape[0], *self.E1.shape),
                            self.E2.expand(price.shape[0], *self.E2.shape),
                            self.eta.expand(price.shape[0], *self.eta.shape), flag=True)
            # plot_y = d.detach().numpy()[0] - p.detach().numpy()[0]
            # axes.plot(plot_y,color='orange')
            # anno = plt.annotate(f'step:{i}', xy=(0.8, 0.9), xycoords='axes fraction',color='black')
            # plt.pause(0.01)
            # axes.clear()
            if ite < 10:
                with torch.no_grad():
                    u = torch.mean(torch.sum((p-d)*price, axis=1)).detach().numpy()
            #d = d.permute(1,0).unsqueeze(2)
            if ite < 10:
                with torch.no_grad():
                    f = self.costNN(d).detach().numpy()
                    self.record.append(np.mean(f+u))
                    
        plt.pause(0)
        
        c2, c1 = self.approximate_cost(d,  diff=True) # c2: [B,T], c1: [B,T]

        return self.layer(
            price,
            c1, 
            c2,
            self.E1.expand(price.shape[0], *self.E1.shape),
            self.E2.expand(price.shape[0], *self.E2.shape),
            self.eta.expand(price.shape[0], *self.eta.shape),
        ) # return: [2, B, T]

In [9]:
df_dp = np.load("dataset/data_N365_0.npz")
df_price = df_dp["price"]

T = 24
N_train = 20
P1 = df_dp["p"].max()
P2 = df_dp["d"].max()
d = df_dp["d"]
p = df_dp["p"]
price_tensor = torch.from_numpy(df_price[0:N_train]).double()
d_tensor = torch.from_numpy(d[0:N_train]).double()
p_tensor = torch.from_numpy(p[0:N_train]).double()
y_tensor = tuple([d_tensor, p_tensor])

torch.manual_seed(0)
layer = DRagent(P1, P2, T)
layer(price_tensor, d_tensor, 0)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x24 and 6x24)

In [4]:
dataset = np.load("../EnergyStorage2/dataset/version2/data_N240_8.npz")
dataset['paras']

array([[ 0.5 ,  0.5 ,  8.69, 10.62,  4.  ,  0.99]])

In [10]:
torch.manual_seed(0)
dataset = np.load("../EnergyStorage2/dataset/version2/data_N240_8.npz")
df_price = dataset["price"]

T = 6
N_train = 40
P1 = dataset["p"].max()
P2 = dataset["d"].max()
d = dataset["d"]
p = dataset["p"]
price_tensor = torch.from_numpy(df_price[10:10+N_train]).double()
d_tensor = torch.from_numpy(d[10:10+N_train]).double()
p_tensor = torch.from_numpy(p[10:10+N_train]).double()
y_tensor = tuple([d_tensor, p_tensor])

price_tensor2 = torch.from_numpy(df_price[:10]).double()
d_tensor2 = torch.from_numpy(d[:10]).double()
p_tensor2 = torch.from_numpy(p[:10]).double()
y_tensor2 = tuple([d_tensor2, p_tensor2])

L = []
val_L = []
layer = DRagent(P1, P2, T)
opt1 = optim.Adam(layer.parameters(), lr=2e-2)
for ite in range(10):
    dp_pred = layer(price_tensor, d_tensor, ite)
    if ite == 10:
        opt1.param_groups[0]["lr"] = 1e-2
    elif ite == 100:
        opt1.param_groups[0]["lr"] = 5e-3
    loss = nn.MSELoss()(y_tensor[0], dp_pred[0]) + nn.MSELoss()(y_tensor[1], dp_pred[1])
    opt1.zero_grad()
    loss.backward()
    opt1.step()
    with torch.no_grad():
        layer.E1.data = torch.clamp(layer.E1.data, min=0.01, max=100) 
        layer.E2.data = torch.clamp(layer.E2.data, min=-100, max=-0.01) 
        layer.eta.data =  torch.clamp(layer.eta.data, min=0.5, max=1) 
        if torch.min(layer.costNN.linear_z1.weight.data) < 1e-2:
            layer.costNN.linear_z1.weight.data = torch.clamp(layer.costNN.linear_z1.weight.data, min=1e-2)
    layer.eval()
    dp_pred2 = layer(price_tensor2, d_tensor2, 500)
    loss2 = nn.MSELoss()(y_tensor2[0], dp_pred2[0]) + nn.MSELoss()(y_tensor2[1], dp_pred2[1])
    val_L.append(loss2.detach().numpy())
    print(f'ite = {ite}, loss = {loss.detach().numpy()}, eta = {layer.eta.data}, val_l = {loss2.detach().numpy()}')
    L.append(loss.detach().numpy())

L = [float(l) for l in L]
val_L = [float(l) for l in val_L]

/Users/bb/opt/anaconda3/envs/dr/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:178: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


ite = 0, loss = 0.06032953013495276, eta = tensor([0.8800]), val_l = 0.07780543084571946
ite = 1, loss = 0.05490032692475479, eta = tensor([0.8624]), val_l = 0.07275824631355166
ite = 2, loss = 0.051488278692721254, eta = tensor([0.8468]), val_l = 0.06884065955032656
ite = 3, loss = 0.04847619063874941, eta = tensor([0.8299]), val_l = 0.06511727594406312
ite = 4, loss = 0.04461677085592913, eta = tensor([0.8143]), val_l = 0.0598304510435735
ite = 5, loss = 0.041714824105118904, eta = tensor([0.7994]), val_l = 0.05594253378837519
ite = 6, loss = 0.03969193158211852, eta = tensor([0.7863]), val_l = 0.053910268339662275
ite = 7, loss = 0.03838771206382923, eta = tensor([0.7752]), val_l = 0.05236925694683975
ite = 8, loss = 0.03729257434171254, eta = tensor([0.7657]), val_l = 0.05160327480063659
ite = 9, loss = 0.03637285817897836, eta = tensor([0.7576]), val_l = 0.05107781887481447


In [12]:
np.array(layer.record).reshape(20,20)

ValueError: cannot reshape array of size 400 into shape (10,20)

In [ ]:
k = 10
import plotly
from plotly.express.colors import sample_colorscale
x = np.linspace(0, 1, 25)
c = sample_colorscale('Greys', list(x))
fig = go.Figure()
for i in range(2,15):
    fig.add_trace(go.Scatter(y=[layer.record[j]/abs(layer.record[i*10]) for j in range(i*10,(i+1)*10)], showlegend=False, line={'color':c[i+5]}))
fig.update_layout({
    'xaxis':{'title': r"k", 'showline':True},
    'yaxis':{'title': r'$c(y^{(k)})/|c(y^{(0)})|$'},
    #'yaxis2': {'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right', 'range':[0,40],'showgrid':False},
    "font": {"size": 20,'color':'#3a4142'},
    'template':'plotly_white',
    'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
    'width':800,
    'height':500,
    'coloraxis': {'colorscale':'viridis'}
})
fig.show()

In [ ]:
fig.write_image("result/images/convergence.pdf")

In [ ]:
pred_d, pred_p = layer(price_tensor, d_tensor, 500)
pred_d = pred_d.detach().numpy()
pred_p = pred_p.detach().numpy()

In [ ]:
d = df_dp["d"]
p = df_dp["p"]
d2 = df_dp["d"][:10]
p2 = df_dp["p"][:10]
price_tensor2 = torch.from_numpy(df_price[:10]).double()
d_tensor2 = torch.from_numpy(d[:10]).double()
p_tensor2 = torch.from_numpy(p[:10]).double()
pred_d2, pred_p2 = layer(price_tensor2, d_tensor2, 500)
pred_d2 = pred_d2.detach().numpy()
pred_p2 = pred_p2.detach().numpy()

In [ ]:
c2, c1 = layer.approximate_cost(d_tensor, diff=True)
torch.mean(torch.diag(c2[10])), torch.mean(c1[10]), layer.E1.data, layer.E2.data, layer.eta.data,

In [ ]:
torch.diag(c2[10])

In [ ]:
torch.save(layer.state_dict(), '../EnergyStorage/result/model_gradient/ICNN_vector/model_gradient_model_0.pth')

In [ ]:
df = pd.read_csv('/Users/bb/Desktop/UCSD/research/E2E/e2e-DR-learning/energystorage-model/Results/OptNet_val_loss.csv')
opt_quad = df.values[:,1:]
opt_quad.shape

In [ ]:
val_loss_mlp = np.zeros((10, 500))
val_loss_rnn = np.zeros((10, 500))
val_loss_model = np.zeros((10, 500))
for i in range(10):
    val_loss_mlp[i] = np.load(f'result/baseline/MLP_loss_{i}.npz')['val_loss']
    val_loss_rnn[i] = np.load(f'result/baseline/RNN_loss_{i}.npz')['val_loss']
for i in range(10):
    val_loss_model[i] = np.load(f'result/model_gradient/loss_{i}.npz')['val_loss']

fig = go.Figure()
x1 = np.linspace(0,500,100)
x2 = np.linspace(0,500,100)
x3 = np.linspace(0,500,6)

median_NN =  np.median(val_loss_mlp, axis=0)[:500]
median_NN_2 = np.quantile(val_loss_mlp,0.2, axis=0)[:500]
median_NN_8 = np.quantile(val_loss_mlp,0.8,axis=0)[:500]

median_RNN =  np.median(val_loss_rnn, axis=0)[:500]
median_RNN_2 = np.quantile(val_loss_rnn,0.2,axis=0)[:500]
median_RNN_8 = np.quantile(val_loss_rnn,0.8,axis=0)[:500]

median_opt0 =  np.load(f'result/model_gradient/ICNN_vector/loss_0.npz')['val_loss']#np.median(opt_quad, axis=0)
median_opt0_2 = np.quantile(opt_quad, 0.2, axis=0)
median_opt0_8 = np.quantile(opt_quad, 0.8, axis=0)

median_opt1 =  np.median(val_loss_model, axis=0)[:500]
median_opt1_2 = np.quantile(val_loss_model, 0.2, axis=0)
median_opt1_8 = np.quantile(val_loss_model, 0.8, axis=0)


fig.add_trace(go.Scatter(x=x1, y=median_NN_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False ))
fig.add_trace(go.Scatter(x=x1,y=median_NN_8,fill='tonexty', mode='lines',  fillcolor='rgba(203,158,252,0.2)',line_color = 'rgba(0,0,0,0)',showlegend = False))

fig.add_trace(go.Scatter(x=x1, y=median_RNN_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False ))
fig.add_trace(go.Scatter(x=x1,y=median_RNN_8,fill='tonexty', mode='lines',  fillcolor='rgba(168,236,217,0.5)',line_color='rgba(0,0,0,0)',showlegend = False))

fig.add_trace(go.Scatter(x=x2, y=median_opt1_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False    ))
fig.add_trace(go.Scatter(x=x2,y=median_opt1_8,fill='tonexty', mode='lines',  fillcolor='rgba(240,96,72,0.5)',line_color = 'rgba(0,0,0,0)',showlegend = False))

fig.add_trace(go.Scatter(x=x3, y=median_opt0_2,fill=None,mode='lines',line_color='rgba(0,0,0,0)', showlegend = False    ))
fig.add_trace(go.Scatter(x=x3,y=median_opt0_8,fill='tonexty', mode='lines',  fillcolor='rgba(250,232,20,0.5)',line_color = 'rgba(0,0,0,0)',showlegend = False))

fig.add_trace(go.Scatter(x=x1,y=median_NN, mode='lines',  line_color='#b681f0',opacity=0.5, line_width=3,name = 'median(MLP,300)')) 
fig.add_trace(go.Scatter(x=x1,y=median_RNN, mode='lines',  line_color='#07cd98',opacity=0.7,line_width=3,name = 'median(RNN,300)')) 
fig.add_trace(go.Scatter(x=x2,y=median_opt1, mode='lines',  line_color='#f06048',opacity=0.5,line_width=3,name = 'median(Ours-nonconvex,20)')) 
fig.add_trace(go.Scatter(x=x1,y=median_opt0, mode='lines',  line_color='#fff50e',opacity=0.5,line_width=3,name = 'median(Ours-convex,20)')) 

fig.update_layout({#'title': 'NRMSE comparison',
    'xaxis':{'title':'Iterations', 'showline':True},
    'yaxis':{'title': 'Validation Loss',"range":[0,0.05]},
    "font": {"size": 40, 'color':'#3a4142'},
    'template':'plotly_white',
    'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
    'width':1600,
   'height':900
})

fig.show()


In [ ]:
fig.write_image("result/images/validation_loss.pdf")

In [ ]:
fig.write_image("result/images/test_1.pdf")

In [ ]:
rnnmodel = keras.models.load_model('result/baseline/model_RNN_1')
mlpmodel = keras.models.load_model('result/baseline/model_MLP_1')

In [ ]:
from differentiable_DR_model import DRagent
dataset = np.load(f"dataset/data_N365_1.npz")
P1 = dataset["p"].max()
P2 = dataset["d"].max()
T = 24
model = DRagent(P1, P2, T, type='vector')
model.load_state_dict(torch.load('result/model_gradient/ICNN_vector/model_1.pth'))
model.eval()

In [ ]:
test_price = dataset['price'][:20]
df_dp = dataset
d2 = df_dp["d"][:20]
p2 = df_dp["p"][:20]
test_dr = p2-d2
price_tensor2 = torch.from_numpy(test_price).double()
d_tensor2 = torch.from_numpy(d2).double()
p_tensor2 = torch.from_numpy(p2).double()
pred_d2, pred_p2 = model(price_tensor2, d_tensor2, 0)
pred_d2 = pred_d2.detach().numpy()
pred_p2 = pred_p2.detach().numpy()
pred_dr = pred_p2-pred_d2

In [ ]:
i = np.random.randint(10)
d = df_dp["d"]
p = df_dp["p"]
df_price = df_dp['price']
test_price = df_price[:10]

dr1 = rnnmodel(df_price[i:i+1])
dr3 = mlpmodel(df_price[i:i+1])
test_dr = p[:10] - d[:10]
pred_dr = pred_p2 - pred_d2 
t1 = go.Scatter(x=np.arange(24), y=test_price[i],mode='lines', name='price',line = {'width':6,'dash':'dash','color':'#ffa05a'},yaxis='y2')
t2 = go.Scatter(x=np.arange(24), y=test_dr[i],mode='lines', name='True',line = {'width':6,'dash':'dash'})
t5 = go.Scatter(x=np.arange(24), y=pred_dr[i],mode='lines', name='Ours(non-convex)',line = {'width':4})
t3 = go.Scatter(x=np.arange(24), y=dr1[0],mode='lines', name='RNN(200)',line = {'width':4})
t6 = go.Scatter(x=np.arange(24), y=dr3[0],mode='lines', name='MLP(200)',line = {'width':4})
fig = go.Figure(data=[t2,t5, t3,t6,t1],layout={
    'xaxis':{'title':'Hour', 'showline':True},
    'yaxis':{'title': 'Dispatch(MW)','range':[-0.6,0.5]},
    'yaxis2': {'title':'Price ($/MWh)','anchor':'x', 'overlaying':'y', 'side':'right','range':[10,100], 'showgrid':False},
    "font": {"size": 40,'color':'#3a4142'},
    'template':'plotly_white',
    'legend':{'yanchor':'bottom','y':1,'xanchor':'left','x':0.0,'orientation':'h'},
    'width':1600,
    'height':900
})
fig.show()

In [ ]:
fig.write_image("result/images/predict.pdf")